In [2]:
import pandas as pd

# Load the xlsx file
cols = [('', '№'),
        ('Трасса 1', 'Широта'),
        ('Трасса 1', 'Долгота'),
        ('Трасса 2', 'Широта'),
        ('Трасса 2', 'Долгота')]

custom_multicols = pd.MultiIndex.from_tuples(cols)
excel_data = pd.read_excel('трасса.xlsx', header = None, names = cols, skiprows= [0] )

#to Frame
data = pd.DataFrame(excel_data)
# Print the content
display(data.head().style.hide_index())


C:\Users\Dmitriy Novichkov\AppData\Local\Temp\ipykernel_8492\4080754483.py:16: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  display(data.head().style.hide_index())


In [3]:
import numpy as np

def get_cartesian(lat=None,lon=None):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    R = 6371 # radius of the earth
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R * np.sin(lat)
    return [x,y]
print(get_cartesian(56.0,30.0))

[3085.3176815979073, 1781.3089940060636]


In [4]:
def get_azimut(pt1, pt2):
    x = pt2[0] - pt1[0]
    y = pt2[1] - pt1[1]
    df = np.rad2deg(np.arctan(abs(pt2[0] - pt1[0]) / abs(pt2[1] - pt1[1])))
    
    if x >= 0 and y >= 0: alp = 0
    elif x > 0 and y < 0: alp = 180
    elif x < 0 and y < 0: alp = 270
    elif x < 0 and y > 0: alp = 360

    return abs(alp - df)

print(get_azimut([0,0],[2,-1]))


116.56505117707799


In [5]:
from dubins_3Dpath_planning import*

print("3D-Dubins Planner Start!!")    

"""
3D-Dubins path plannner

input:
    start_x : x position of start point [m]
    start_y : y position of start point [m]
    start_z : z position of start point [m]
    start_psi : heading angle of start point [rad]
    start_gamma : flight path angle of start point [rad]
    
    end_x : x position of end point [m]
    end_y : y position of end point [m]
    end_z : z position of end point [m]
    end_psi : heading angle of end point [rad]
    end_gamma : flight path angle of end point [rad]
    
    Rmin : minimum turning radius [m]

output:
    px : x coordinates of path
    py : y coordinates of path
    pz : z coordinates of path
    ppsi : heading angle of path points
    pgamma: flight path angle of path points
    mode : type of curve

"""

#    start_x = 0
#    start_y = 2
#    start_z = -1
#    start_psi = np.deg2rad(10.0)
#    start_gamma = np.deg2rad(-50.0) 
#    
#    end_x = 4
#    end_y = 1
#    end_z = -2
#    end_psi = np.deg2rad(-10.0)
#    end_gamma = np.deg2rad(-20.0)

Rmin = 1        
flag = 1


try:
    px, py, pz, ppsi, pgamma, clen, mode = dubins_3Dpath_planning(start_x, start_y, start_z, start_psi, start_gamma,
                                                    end_x, end_y, end_z, end_psi, end_gamma, Rmin)
    #print("Trajectory Type ---->",mode," and Length ----->",clen)
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot3D(px, py, pz, 'black')
    plt.show()
except:
    print("NOT POSSIBLE")



Wptz = []

x0 = data['Трасса 1']['Широта'][0]
y0 = data['Трасса 1']['Долгота'][0]
x0, y0 = get_cartesian(x0, y0)

for colum in range(len(data.values) // 2):
x = data['Трасса 1']['Широта'][colum]
y = data['Трасса 1']['Долгота'][colum]
x, y = get_cartesian(x, y)
x2 = data['Трасса 1']['Широта'][colum + 1]
y2 = data['Трасса 1']['Долгота'][colum + 1]
x2, y2 = get_cartesian(x2, y2)
azimut = get_azimut([x,y],[x2,y2])
x3 = data['Трасса 1']['Широта'][colum + 2]
y3 = data['Трасса 1']['Долгота'][colum + 2]
x3, y3 = get_cartesian(x3, y3)
azimut2 = get_azimut([x2,y2],[x3,y3])
Wptz.append(Waypoint(1000*(x - x0), 1000*(y - y0), azimut))
Wptz.append(Waypoint(1000*(x2 - x0), 1000*(y2 - y0), azimut2))

if len(data.values) % 2 == 1:
x2 = data['Трасса 1']['Широта'][len(data.values) - 3]
y2 = data['Трасса 1']['Долгота'][len(data.values) - 3]
x = data['Трасса 1']['Широта'][len(data.values) - 2]
y = data['Трасса 1']['Долгота'][len(data.values) - 2]
x, y = get_cartesian(x, y)
azimut = get_azimut([x2,y2],[x,y])
Wptz.append(Waypoint(1000*(x - x0), 1000*(y - y0), azimut))

print(len(Wptz))

# Run the code
i = 0
while i<len(Wptz)-1:
param = calcDubinsPath(Wptz[i], Wptz[i+1], 20, 30)
path = dubins_traj(param,1)
# Plot the results
plt.plot3D(Wptz[i].x,Wptz[i].y,'kx')
plt.plot3D(Wptz[i+1].x,Wptz[i+1].y,'kx')
plt.plot3D(path[:,0],path[:,1],10*path[:,2],'b-')
i+=1

plt.grid(True)
plt.axis("equal")
#plt.title('Dubin\'s Curves Trajectory Generation')
#plt.xlabel('X')
#plt.ylabel('Y')
#plt.show()


IndentationError: expected an indented block after 'for' statement on line 68 (2800318787.py, line 69)